This notebook was created to support the data preparation required to support our CS 598 DLH project.  The paper we have chosen for the reproducibility project is:
***Ensembling Classical Machine Learning and Deep Learning Approaches for Morbidity Identification from Clinical Notes ***

This notebook is for creating the multiple embeddings formats as described in the study.

 

The data cannot be shared publicly due to the agreements required to obtain the data so we are storing the data locally and not putting in GitHub.

We are only creating embeddings for data that includes stop words.

In [5]:
pip install torchtext

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import numpy as np

DATA_PATH = './obesity_data/'
test_df = pd.read_pickle(DATA_PATH + '/test_df.pkl')
train_df = pd.read_pickle(DATA_PATH + '/train_df.pkl')
test_annot_all_df_clean = pd.read_pickle(DATA_PATH + '/test_annot_all_df_clean.pkl') 
train_annot_all_df_clean = pd.read_pickle(DATA_PATH + '/train_annot_all_df_clean.pkl') 


test_df['word_count'] = test_df['lower_text'].apply(lambda x: len(x.split()))
train_df['word_count'] = train_df['lower_text'].apply(lambda x: len(x.split()))

df_print = pd.DataFrame()
df_print['Min'] = [np.min(test_df['word_count']), np.min(train_df['word_count'])]
df_print['Mean'] = [np.mean(test_df['word_count']), np.mean(train_df['word_count'])]
df_print['Max'] = [np.max(test_df['word_count']), np.max(train_df['word_count'])]
df_print['Std'] = [np.std(test_df['word_count']), np.std(train_df['word_count'])]
df_print['MeanPlusStd'] = round(df_print['Mean'] + df_print['Std'],0)
token_max = int(round(np.max(df_print['MeanPlusStd']),0))

print(df_print)
print('Max Tokens:',token_max)
print('Test:', sum(test_df['word_count'] > token_max), "out of", len(test_df))
print('Train:', sum(train_df['word_count'] > token_max), "out of", len(train_df))


   Min        Mean   Max         Std  MeanPlusStd
0  206  991.907298  3124  437.621770       1430.0
1  113  958.774141  3748  444.819254       1404.0
Max Tokens: 1430
Test: 79 out of 507
Train: 72 out of 611


We are going to split these larger text blocks into 2 notes of size max_token or below.  Note, there are 4 notes (1 in test and 3 in train) that are bigger than 2 times x tokens.  For now, we will ignore, but may want to add in later (either loop or have left/middle/right).

In [7]:
test_df_ok = test_df[test_df['word_count'] <= token_max].copy()
test_df_large_right = test_df[test_df['word_count'] > token_max].copy()
test_df_large_left = test_df[test_df['word_count'] > token_max].copy()

train_df_ok = train_df[train_df['word_count'] <= token_max].copy()
train_df_large_right = train_df[train_df['word_count'] > token_max].copy()
train_df_large_left = train_df[train_df['word_count'] > token_max].copy()

#Get the right words and the left words and then concatenate all 3 and recacluate 
test_df_large_left['lower_text'] = test_df_large_left['lower_text'].apply(lambda x: ' '.join([word for word in x.split()[:(token_max-1)]]))
test_df_large_right['lower_text'] = test_df_large_right['lower_text'].apply(lambda x: ' '.join([word for word in x.split()[token_max:(2*token_max)]]))
train_df_large_left['lower_text'] = train_df_large_left['lower_text'].apply(lambda x: ' '.join([word for word in x.split()[:(token_max-1)]]))
train_df_large_right['lower_text'] = train_df_large_right['lower_text'].apply(lambda x: ' '.join([word for word in x.split()[token_max:(2*token_max)]]))

test_df_expanded = pd.concat([test_df_ok,test_df_large_right,test_df_large_left])
test_df_expanded['word_count'] = test_df_expanded['lower_text'].apply(lambda x: len(x.split()))
train_df_expanded = pd.concat([train_df_ok,train_df_large_right,train_df_large_left])
train_df_expanded['word_count'] = train_df_expanded['lower_text'].apply(lambda x: len(x.split()))

print('Test:', sum(test_df_expanded['word_count'] > token_max), "out of", len(test_df_expanded))
print('Train:', sum(train_df_expanded['word_count'] > token_max), "out of", len(train_df_expanded))

Test: 0 out of 586
Train: 0 out of 683


We need to create a one hot vector given a vocabulary and pad it with the padding character.

In [17]:
from typing import Union, Iterable
import torchtext, torch, torch.nn.functional as F
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

#shouldn't have to do this, need to see why lower_text is removing a bunch of data
test_df_expanded["no_numerics_text"] = test_df_expanded['no_numerics_text'].apply(str.lower)
train_df_expanded["no_numerics_text"] = train_df_expanded['no_numerics_text'].apply(str.lower)

#corpus = pd.concat([test_df_expanded['lower_text'],train_df_expanded['lower_text']])
corpus = pd.concat([test_df_expanded['no_numerics_text'],train_df_expanded['no_numerics_text']])
tokenizer = get_tokenizer("basic_english")
tokens = [tokenizer(doc) for doc in corpus]

voc = build_vocab_from_iterator(tokens, specials = ['<pad>'])

#so need to create one hot encoding but add <pad> to reach max_tokens
#need to do for both stop words and non stop words???
def encode_and_pad(vocab, input_tokens, token_max):
    pad_zeros = token_max - len(input_tokens)
    result = vocab.lookup_indices(input_tokens)
    if pad_zeros > 0:
        result.extend(np.zeros(pad_zeros, dtype=int))
    return result

#train_df_expanded['one_hot'] = train_df_expanded['lower_text'].apply(lambda x: encode_and_pad(voc, x.split(), token_max))
#test_df_expanded['one_hot'] = test_df_expanded['lower_text'].apply(lambda x: encode_and_pad(voc, x.split(), token_max))
train_df_expanded['one_hot'] = train_df_expanded['no_numerics_text'].apply(lambda x: encode_and_pad(voc, x.split(), token_max))
test_df_expanded['one_hot'] = test_df_expanded['no_numerics_text'].apply(lambda x: encode_and_pad(voc, x.split(), token_max))

print(train_df_expanded.iloc[0])



id                                                                  4
text                368346277 | EMH | 64927307 | | 815098 | 3/29/1...
no_punc_text        368346277  EMH  64927307   815098  3291993 120...
no_numerics_text      emh         am  discharge summary  signed  d...
lower_text          emh am discharge summary signed dis admission ...
word_count                                                        413
one_hot             [7366, 78, 20, 147, 131, 152, 25, 60, 119, 38,...
Name: 2, dtype: object


Join the test data documents with their associated annotations.  Verify the number of records are the same.

In [18]:
test_with_annot_df = pd.merge(test_annot_all_df_clean,test_df, on='id')
train_with_annot_df = pd.merge(train_annot_all_df_clean,train_df, on='id')


test_with_annot_df_expanded = pd.merge(test_annot_all_df_clean,test_df_expanded,on='id')
train_with_annot_df_expanded = pd.merge(train_annot_all_df_clean,train_df_expanded,on='id')



print("Test:", len(test_annot_all_df_clean), len(test_df), len(test_with_annot_df))
print("Train:", len(train_annot_all_df_clean), len(train_df), len(train_with_annot_df))

print("Test Expanded:", len(test_annot_all_df_clean), len(test_df_expanded), len(test_with_annot_df_expanded))
print("Train Expanded:", len(train_annot_all_df_clean), len(train_df_expanded), len(train_with_annot_df_expanded))



Test: 7542 507 7542
Train: 8783 611 8783
Test Expanded: 7542 586 8701
Train Expanded: 8783 683 9809


Try and validate the numbers are close with the original papers.  You can see the counts are higher for some reason but the percentage occurrence of each disease doesn't change too much so we are good to use the expanded set.

In [19]:
all_df = pd.concat([test_with_annot_df, train_with_annot_df])
all_df_extended = pd.concat([test_with_annot_df_expanded, train_with_annot_df_expanded])

df_before = pd.concat([all_df['disease'].value_counts().sort_index(0),all_df[all_df['judgment']==True]['disease'].value_counts().sort_index(0)/all_df['disease'].value_counts().sort_index(0)],axis =1)
df_after = pd.concat([all_df_extended['disease'].value_counts().sort_index(0),all_df_extended[all_df_extended['judgment']==True]['disease'].value_counts().sort_index(0)/all_df_extended['disease'].value_counts().sort_index(0)],axis =1)
df_all = pd.concat([df_before,df_after], axis=1)

df_all


C:\Users\chris\AppData\Local\Temp\ipykernel_29108\1675168617.py:4: FutureWarning: In a future version of pandas all arguments of Series.sort_index will be keyword-only.
  df_before = pd.concat([all_df['disease'].value_counts().sort_index(0),all_df[all_df['judgment']==True]['disease'].value_counts().sort_index(0)/all_df['disease'].value_counts().sort_index(0)],axis =1)
C:\Users\chris\AppData\Local\Temp\ipykernel_29108\1675168617.py:4: FutureWarning: In a future version of pandas all arguments of Series.sort_index will be keyword-only.
  df_before = pd.concat([all_df['disease'].value_counts().sort_index(0),all_df[all_df['judgment']==True]['disease'].value_counts().sort_index(0)/all_df['disease'].value_counts().sort_index(0)],axis =1)
C:\Users\chris\AppData\Local\Temp\ipykernel_29108\1675168617.py:4: FutureWarning: In a future version of pandas all arguments of Series.sort_index will be keyword-only.
  df_before = pd.concat([all_df['disease'].value_counts().sort_index(0),all_df[all_df['ju

,disease,disease,disease,disease
Asthma,1057,0.142857,1195,0.143933
CAD,1044,0.606322,1187,0.608256
CHF,723,0.672199,838,0.693317
Depression,1068,0.220974,1211,0.230388
Diabetes,1070,0.702804,1216,0.718750
GERD,924,0.239177,1036,0.247104
Gallstones,1097,0.164084,1244,0.171222
Gout,1102,0.131579,1250,0.136800
Hypercholesterolemia,961,0.548387,1088,0.553309
Hypertension,1037,0.812922,1177,0.816483


![Note occurrences](images\note_occurrences.gif)


Save the final test/train dataset

In [20]:
test_with_annot_df_expanded.to_pickle(DATA_PATH + '/test.pkl') 
train_with_annot_df_expanded.to_pickle(DATA_PATH + '/train.pkl') 
corpus.to_pickle(DATA_PATH + '/corpus.pkl')

Testing the GloVe embedding

In [21]:
from torchtext.vocab import vocab


vec = torchtext.vocab.GloVe(name='6B', dim=300)

one_hot_test = train_df_expanded.iloc[0]['one_hot']

print(one_hot_test)
ret = vec.get_vecs_by_tokens(voc.lookup_tokens(one_hot_test))
print(ret.shape)

[7366, 78, 20, 147, 131, 152, 25, 60, 119, 38, 131, 20, 60, 5344, 105, 87, 77, 54, 107, 693, 464, 400, 54, 123, 176, 16, 279, 511, 176, 26, 3, 182, 236, 1, 12, 18, 8, 442, 429, 721, 23121, 50, 30190, 3315, 7, 8, 864, 26, 3, 514, 19, 23, 99, 264, 11, 1, 352, 5289, 4682, 10, 393, 7, 3584, 2626, 10, 2698, 402, 3, 22, 87, 77, 54, 1, 12, 23, 1251, 2, 23, 99, 507, 189, 1721, 13944, 3, 1, 514, 2406, 677, 22, 514, 23, 2069, 2, 19, 2024, 5, 2021, 343, 793, 722, 19, 4, 1690, 10, 1624, 71, 214, 100, 90, 26, 366, 10, 29, 527, 3, 41, 27, 11, 61, 123, 2, 26, 3, 464, 400, 54, 7, 2392, 194, 148, 134, 6, 148, 161, 1, 74, 319, 4, 56, 80, 505, 674, 2, 370, 161, 931, 406, 196, 3518, 56, 142, 80, 2, 132, 16, 712, 1492, 239, 290, 451, 206, 16, 117, 23, 31056, 2373, 386, 3, 405, 22, 148, 161, 4, 422, 6, 22, 3017, 25, 5, 1, 71, 153, 380, 66, 3607, 1116, 17, 1624, 71, 214, 332, 134, 22, 25, 332, 161, 4, 1435, 10, 8, 68, 1350, 2, 1544, 494, 161, 22, 157, 66, 71, 214, 574, 3074, 133, 1379, 11, 972, 665, 2, 187, 

Testing the FastText embedding.

In [22]:
from torchtext.vocab import vocab

vec = torchtext.vocab.FastText()

one_hot_test = train_df_expanded.iloc[0]['one_hot']

print(one_hot_test)
ret = vec.get_vecs_by_tokens(voc.lookup_tokens(one_hot_test))
print(ret.shape)

[7366, 78, 20, 147, 131, 152, 25, 60, 119, 38, 131, 20, 60, 5344, 105, 87, 77, 54, 107, 693, 464, 400, 54, 123, 176, 16, 279, 511, 176, 26, 3, 182, 236, 1, 12, 18, 8, 442, 429, 721, 23121, 50, 30190, 3315, 7, 8, 864, 26, 3, 514, 19, 23, 99, 264, 11, 1, 352, 5289, 4682, 10, 393, 7, 3584, 2626, 10, 2698, 402, 3, 22, 87, 77, 54, 1, 12, 23, 1251, 2, 23, 99, 507, 189, 1721, 13944, 3, 1, 514, 2406, 677, 22, 514, 23, 2069, 2, 19, 2024, 5, 2021, 343, 793, 722, 19, 4, 1690, 10, 1624, 71, 214, 100, 90, 26, 366, 10, 29, 527, 3, 41, 27, 11, 61, 123, 2, 26, 3, 464, 400, 54, 7, 2392, 194, 148, 134, 6, 148, 161, 1, 74, 319, 4, 56, 80, 505, 674, 2, 370, 161, 931, 406, 196, 3518, 56, 142, 80, 2, 132, 16, 712, 1492, 239, 290, 451, 206, 16, 117, 23, 31056, 2373, 386, 3, 405, 22, 148, 161, 4, 422, 6, 22, 3017, 25, 5, 1, 71, 153, 380, 66, 3607, 1116, 17, 1624, 71, 214, 332, 134, 22, 25, 332, 161, 4, 1435, 10, 8, 68, 1350, 2, 1544, 494, 161, 22, 157, 66, 71, 214, 574, 3074, 133, 1379, 11, 972, 665, 2, 187, 